#### 从数据库导入数据

In [1]:
import pymysql.cursors
import pandas as pd

db = pymysql.connect(host='localhost',user='root',password='root',db='lagou',charset='utf8',cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()
sql = 'SELECT * FROM dataset;'
cursor.execute(sql)
result = cursor.fetchall()
df = pd.DataFrame(list(result))
db.close()

df

,jobNature,companySize,workYear,education,industryField,thirdType,salaryMin,salaryMax,nameUnfit,positionId,...,financeStage,firstType,secondType,district,salary,resumeProcessRate,resumeProcessDay,score,famousCompany,isFocus
0,全职,500-2000人,1-3年,本科,旅游,数据分析,18,35,0,6112946,...,D轮及以上,开发|测试|运维类,数据开发,福田区,18k-35k,100,1,199,0,1
1,全职,500-2000人,3-5年,本科,"消费生活,移动互联网",商业,15,25,0,6229664,...,C轮,产品|需求|项目类,数据分析,南山区,15k-25k,100,1,179,0,1
2,全职,2000人以上,5-10年,本科,金融,建模,15,30,0,6853427,...,上市公司,开发|测试|运维类,数据开发,福田区,15k-30k,100,1,130,1,0
3,全职,2000人以上,1-3年,本科,金融,Hadoop,20,30,0,6692363,...,不需要融资,开发|测试|运维类,数据开发,南山区,20k-30k,100,2,89,0,0
4,全职,2000人以上,1-3年,大专,"移动互联网,游戏",数据分析,8,10,0,6835783,...,不需要融资,产品|需求|项目类,数据分析,南山区,8k-10k,0,1,69,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,全职,2000人以上,3-5年,本科,社交,产品经理,15,25,1,5276304,...,上市公司,产品|需求|项目类,产品经理,南山区,15k-25k,100,1,10,1,0
271,全职,2000人以上,3-5年,本科,社交,数据挖掘,18,35,0,6842800,...,上市公司,开发|测试|运维类,数据开发,南山区,18k-35k,100,1,10,1,1
272,全职,2000人以上,3-5年,本科,社交,数据分析,20,40,0,5250466,...,上市公司,产品|需求|项目类,数据分析,南山区,20k-40k,100,1,10,1,1
273,全职,2000人以上,1-3年,本科,金融,数据分析,20,30,0,6391267,...,未融资,开发|测试|运维类,数据开发,南山区,20k-30k,15,1,4,1,0


#### 拆分训练集和测试集

In [2]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
X = pd.get_dummies(X).values
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
X_names = pd.get_dummies(df.iloc[:,:-1]).columns
X_names

Index(['salaryMin', 'salaryMax', 'nameUnfit', 'resumeProcessRate',
       'resumeProcessDay', 'score', 'famousCompany', 'jobNature_全职',
       'jobNature_兼职', 'jobNature_实习',
       ...
       'salary_7k-13k', 'salary_7k-14k', 'salary_8k-10k', 'salary_8k-11k',
       'salary_8k-12k', 'salary_8k-13k', 'salary_8k-15k', 'salary_8k-16k',
       'salary_9k-12k', 'salary_9k-14k'],
      dtype='object', length=687)

#### 网格搜索（含交叉验证）——找出最优参数

In [8]:
?GridSearchCV

### 一、SVC

In [10]:
?SVC

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

SVC = SVC()
parameters = {
    'random_state':[0],
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma':['auto', 'scale']
}
scorings = ['recall','roc_auc']
for scoring in scorings:
    print('===========================================================================')
    print('此次调参的scoring为：%s' % scoring)
    print()
    clf = GridSearchCV(SVC, parameters, scoring=scoring, cv=10)
    clf.fit(X_train, y_train)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    print('【交叉验证】过程如下：')
    for mean, std, param in zip(means, stds, params):
        print('%s得分为 %0.3f(+-%0.3f)时，参数为：%r' % (scoring, mean, std * 2, param))
    print()
    print('【训练集】最佳参数为：%r' % clf.best_params_)
    print('【训练集】最佳参数得分为：%0.3f' % clf.best_score_)
    print()
    print('【测试集】%s得分为：%0.3f' % (scoring, clf.score(X_test, y_test)))    # 直接获取测试集的得分
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    reports = classification_report(y_true, y_pred)
    print('【测试集中实际值与预测值对比】模型报告如下：')
    print(reports)

此次调参的scoring为：recall



C:\Users\romak\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


【交叉验证】过程如下：
recall得分为 0.878(+-0.411)时，参数为：{'gamma': 'auto', 'kernel': 'linear', 'random_state': 0}
recall得分为 0.422(+-0.359)时，参数为：{'gamma': 'auto', 'kernel': 'poly', 'random_state': 0}
recall得分为 0.186(+-0.338)时，参数为：{'gamma': 'auto', 'kernel': 'rbf', 'random_state': 0}
recall得分为 0.205(+-0.311)时，参数为：{'gamma': 'auto', 'kernel': 'sigmoid', 'random_state': 0}
recall得分为 0.878(+-0.411)时，参数为：{'gamma': 'scale', 'kernel': 'linear', 'random_state': 0}
recall得分为 0.041(+-0.162)时，参数为：{'gamma': 'scale', 'kernel': 'poly', 'random_state': 0}
recall得分为 0.000(+-0.000)时，参数为：{'gamma': 'scale', 'kernel': 'rbf', 'random_state': 0}
recall得分为 0.192(+-0.445)时，参数为：{'gamma': 'scale', 'kernel': 'sigmoid', 'random_state': 0}

【训练集】最佳参数为：{'gamma': 'auto', 'kernel': 'linear', 'random_state': 0}
【训练集】最佳参数得分为：0.878

【测试集】recall得分为：0.667

【测试集中实际值与预测值对比】模型报告如下：
              precision    recall  f1-score   support

           0       0.89      0.94      0.91        67
           1       0.80      0.67      0.73        24

C:\Users\romak\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


如果kernel参数含有'precomputed'，则会报错：ValueError: X should be a square kernel matrix

此错误解决方法：https://stackoverflow.com/questions/36306555/scikit-learn-grid-search-with-svm-regression

### 二、LinearSVC

In [17]:
?LinearSVC

In [23]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

lsvc = LinearSVC()
parameters = {
    'random_state':[0],
    #'penalty':['l1', 'l2'],
    'max_iter':[10000],
    'loss':['hinge', 'squared_hinge']
}
scorings = ['recall','roc_auc']
for scoring in scorings:
    print('===========================================================================')
    print('此次调参的scoring为：%s' % scoring)
    print()
    clf = GridSearchCV(lsvc, parameters, scoring=scoring, cv=10)
    clf.fit(X_train, y_train)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    params = clf.cv_results_['params']
    print('【交叉验证】过程如下：')
    for mean, std, param in zip(means, stds, params):
        print('%s得分为 %0.3f(+-%0.3f)时，参数为：%r' % (scoring, mean, std * 2, param))
    print()
    print('【训练集】最佳参数为：%r' % clf.best_params_)
    print('【训练集】最佳参数得分为：%0.3f' % clf.best_score_)
    print()
    print('【测试集】%s得分为：%0.3f' % (scoring, clf.score(X_test, y_test)))    # 直接获取测试集的得分
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    reports = classification_report(y_true, y_pred)
    print('【测试集中实际值与预测值对比】模型报告如下：')
    print(reports)

此次调参的scoring为：recall



C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", 

C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", 

【交叉验证】过程如下：
recall得分为 0.838(+-0.394)时，参数为：{'loss': 'hinge', 'max_iter': 1000, 'random_state': 0}
recall得分为 0.817(+-0.489)时，参数为：{'loss': 'hinge', 'max_iter': 5000, 'random_state': 0}
recall得分为 0.837(+-0.433)时，参数为：{'loss': 'hinge', 'max_iter': 10000, 'random_state': 0}
recall得分为 0.818(+-0.454)时，参数为：{'loss': 'squared_hinge', 'max_iter': 1000, 'random_state': 0}
recall得分为 0.859(+-0.476)时，参数为：{'loss': 'squared_hinge', 'max_iter': 5000, 'random_state': 0}
recall得分为 0.899(+-0.323)时，参数为：{'loss': 'squared_hinge', 'max_iter': 10000, 'random_state': 0}

【训练集】最佳参数为：{'loss': 'squared_hinge', 'max_iter': 10000, 'random_state': 0}
【训练集】最佳参数得分为：0.899

【测试集】recall得分为：0.625

【测试集中实际值与预测值对比】模型报告如下：
              precision    recall  f1-score   support

           0       0.88      0.96      0.91        67
           1       0.83      0.62      0.71        24

    accuracy                           0.87        91
   macro avg       0.86      0.79      0.81        91
weighted avg       0.87      0.87      

C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", 

C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", 

【交叉验证】过程如下：
roc_auc得分为 0.986(+-0.036)时，参数为：{'loss': 'hinge', 'max_iter': 1000, 'random_state': 0}
roc_auc得分为 0.992(+-0.023)时，参数为：{'loss': 'hinge', 'max_iter': 5000, 'random_state': 0}
roc_auc得分为 0.993(+-0.024)时，参数为：{'loss': 'hinge', 'max_iter': 10000, 'random_state': 0}
roc_auc得分为 0.992(+-0.023)时，参数为：{'loss': 'squared_hinge', 'max_iter': 1000, 'random_state': 0}
roc_auc得分为 0.992(+-0.023)时，参数为：{'loss': 'squared_hinge', 'max_iter': 5000, 'random_state': 0}
roc_auc得分为 0.992(+-0.027)时，参数为：{'loss': 'squared_hinge', 'max_iter': 10000, 'random_state': 0}

【训练集】最佳参数为：{'loss': 'hinge', 'max_iter': 10000, 'random_state': 0}
【训练集】最佳参数得分为：0.993

【测试集】roc_auc得分为：0.955

【测试集中实际值与预测值对比】模型报告如下：
              precision    recall  f1-score   support

           0       0.88      0.96      0.91        67
           1       0.83      0.62      0.71        24

    accuracy                           0.87        91
   macro avg       0.86      0.79      0.81        91
weighted avg       0.87      0.87      0

C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\romak\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
